In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
n_samples = mnist.train.num_examples

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
def xavier_init(fan_in, fan_out, constant=1):
    low = -constant*np.sqrt(6.0/(fan_in + fan_out))
    high = constant*np.sqrt(6.0/(fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out), minval=low, maxval=high, dtype=tf.float32)
w, n_input, n_z = {}, 784, 20
n_hidden_recog_1, n_hidden_recog_2 = 500, 500
n_hidden_gener_1, n_hidden_gener_2 = 500, 500
w['w_recog'] = {
    'h1': tf.Variable(xavier_init(n_input, n_hidden_recog_1)),
    'h2': tf.Variable(xavier_init(n_hidden_recog_1, n_hidden_recog_2)),
    'out_mean': tf.Variable(xavier_init(n_hidden_recog_2, n_z)),
    'out_log_sigma': tf.Variable(xavier_init(n_hidden_recog_2, n_z))
}
w['b_recog'] = {
    'b1': tf.Variable(tf.zeros([n_hidden_recog_1], dtype=tf.float32)),
    'b2': tf.Variable(tf.zeros([n_hidden_recog_2], dtype=tf.float32)),
    'out_mean': tf.Variable(tf.zeros([n_z], dtype=tf.float32)),
    'out_log_sigma': tf.Variable(tf.zeros([n_z], dtype=tf.float32))
}
w['w_gener'] = {
    'h1': tf.Variable(xavier_init(n_z, n_hidden_gener_1)),
    'h2': tf.Variable(xavier_init(n_hidden_gener_1, n_hidden_gener_2)),
    'out_mean': tf.Variable(xavier_init(n_hidden_gener_2, n_input)),
    'out_log_sigma': tf.Variable(xavier_init(n_hidden_gener_2, n_input))
}
w['b_gener'] = {
    'b1': tf.Variable(tf.zeros([n_hidden_gener_1], dtype=tf.float32)),
    'b2': tf.Variable(tf.zeros([n_hidden_gener_2], dtype=tf.float32)),
    'out_mean': tf.Variable(tf.zeros([n_input], dtype=tf.float32)),
    'out_log_sigma': tf.Variable(tf.zeros([n_input], dtype=tf.float32))
}

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
l_rate = 0.001
batch_size = 100
x = tf.placeholder(tf.float32, [None, n_input])
enc_layer_1 = tf.nn.softplus(tf.add(tf.matmul(x, w["w_recog"]['h1']), w["b_recog"]['b1']))
enc_layer_2 = tf.nn.softplus(tf.add(tf.matmul(enc_layer_1, w["w_recog"]['h2']), w["b_recog"]['b2']))
z_mean = tf.add(tf.matmul(enc_layer_2, w["w_recog"]['out_mean']), w["b_recog"]['out_mean'])
z_log_sigma_sq = tf.add(tf.matmul(enc_layer_2, w["w_recog"]['out_log_sigma']), w["b_recog"]['out_log_sigma'])

In [5]:
eps = tf.random_normal((batch_size, n_z), 0, 1, dtype=tf.float32)
z = tf.add(z_mean, tf.multiply(tf.sqrt(tf.exp(z_log_sigma_sq)), eps))
dec_layer_1 = tf.nn.softplus(tf.add(tf.matmul(z, w["w_gener"]['h1']), w["b_gener"]['b1']))
dec_layer_2 = tf.nn.softplus(tf.add(tf.matmul(dec_layer_1, w["w_gener"]['h2']), w["b_gener"]['b2']))

x_reconstr_mean = tf.nn.sigmoid(tf.add(tf.matmul(dec_layer_2, w["w_gener"]['out_mean']), w["b_gener"]['out_mean']))
reconstr_loss = -tf.reduce_sum(x * tf.log(1e-10 + x_reconstr_mean) + (1-x) * tf.log(1e-10 + 1 - x_reconstr_mean), 1)
latent_loss = -0.5 * tf.reduce_sum(1 + z_log_sigma_sq - tf.square(z_mean) - tf.exp(z_log_sigma_sq), 1)
cost = tf.reduce_mean(reconstr_loss + latent_loss)
optimizer = tf.train.AdamOptimizer(learning_rate=l_rate).minimize(cost)

Instructions for updating:
Use tf.cast instead.


In [11]:
def train(sess, batch_size=100, training_epochs=10, display_step=5):
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(n_samples / batch_size)
        for i in range(total_batch):
            xs, _ = mnist.train.next_batch(batch_size)
            _, c = sess.run((optimizer, cost), feed_dict={x:xs})
            avg_cost += c / n_samples * batch_size
        if epoch % display_step == 0:
            print("Epoch: %04d\tcost: %.9f" % (epoch+1, avg_cost))
            
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)

train(sess, training_epochs=100, batch_size=batch_size)

Epoch: 0001	cost: 176.293132213
Epoch: 0006	cost: 109.024912109
Epoch: 0011	cost: 104.532355777
Epoch: 0016	cost: 102.339652627
Epoch: 0021	cost: 100.485887978
Epoch: 0026	cost: 99.411555592
Epoch: 0031	cost: 98.611135559
Epoch: 0036	cost: 97.938391821
Epoch: 0041	cost: 97.225544059
Epoch: 0046	cost: 96.849130069
Epoch: 0051	cost: 96.433496441
Epoch: 0056	cost: 95.963860002
Epoch: 0061	cost: 95.867096155
Epoch: 0066	cost: 95.423062494
Epoch: 0071	cost: 95.203929679
Epoch: 0076	cost: 94.997248896
Epoch: 0081	cost: 94.829659646
Epoch: 0086	cost: 94.555005008
Epoch: 0091	cost: 94.438319813
Epoch: 0096	cost: 94.268894861


In [16]:
x_sample = mnist.test.next_batch(100)[0]
x_logits = sess.run(x_reconstr_mean, feed_dict={x: x_sample, eps:np.random.normal(loc=0., scale=1., size=(batch_size, n_z))})
gen_logits = sess.run(x_reconstr_mean, feed_dict={z_mean:np.random.normal(loc=0., scale=1., size=(batch_size, n_z))})

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,784]
	 [[node Placeholder (defined at <ipython-input-4-94ac7990f6dc>:3) ]]

Caused by op 'Placeholder', defined at:
  File "C:\Users\Avsenin.Pavel\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Avsenin.Pavel\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Avsenin.Pavel\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Avsenin.Pavel\AppData\Local\Continuum\anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Avsenin.Pavel\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "C:\Users\Avsenin.Pavel\A

In [17]:
import matplotlib.pyplot as plt
xs, ys = mnist.test.next_batch(5000)
z_mu = sess.run(z_mean, feed_dict={x:xs})
plt.figure(figsize=(8, 6))
plt.scatter(z_mu[:, 0], z_mu[:, 1], c=np.argmax(y_sample, 1))
plt.colorbar()

NameError: name 'y_sample' is not defined

<Figure size 576x432 with 0 Axes>

In [18]:
nx = ny = 20
x_values = np.linspace(-3, 3, nx)
y_values = np.linspace(-3, 3, ny)

canvas = np.empty((28*ny, 28*nx))
for i, yi in enumerate(x_values):
    for j, xi in enumerate(y_values):
        z_mu = np.array([[xi, yi]])
        x_mean = sess.run(x_reconstr_mean, feed_dict={z:z_mu})
        canvas[(nx-i-1)*28:(nx-i)*28, j*28:(j+1)*28] = x_mean[0].reshape(28, 28)
plt.figure(figsize=(8,10))
Xi, Yi = np.meshgrid(x_values, y_values)
plt.imshow(canvas, origin="upper")
plt.tight_layout()

ValueError: Cannot feed value of shape (1, 2) for Tensor 'Add_4:0', which has shape '(100, 20)'

In [19]:
y = tf.placeholder(tf.float32, [None, 10])
xy = tf.concat([x, y], 1)
enc_layer_1 = tf.nn.tanh(tf.add(tf.matmul(xy, w["w_recog"]['h1']), w["b_recog"]['b1']))
zy_mean = tf.concat([z_mean, y], 1)
zy = tf.concat([z, y], 1)
dec_layer_1 = tf.nn.tanh(tf.add(tf.matmul(zy, w["w_gener"]['h1']), w["b_gener"]['b1']))
mean_dec_layer_1 = tf.nn.tanh(tf.add(tf.matmul(zy_mean, w["w_gener"]['h1']), w["b_gener"]['b1']))

ValueError: Dimensions must be equal, but are 794 and 784 for 'MatMul_7' (op: 'MatMul') with input shapes: [?,794], [784,500].